In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
df_sample = pd.read_csv('./data/driving_log.csv')
display(df_sample.head(n=5))

,center,left,right,steering,throttle,brake,speed
0,data/IMG/center_2016_12_01_13_30_48_287.jpg,data/IMG/left_2016_12_01_13_30_48_287.jpg,data/IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,data/IMG/center_2016_12_01_13_30_48_404.jpg,data/IMG/left_2016_12_01_13_30_48_404.jpg,data/IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,data/IMG/center_2016_12_01_13_31_12_937.jpg,data/IMG/left_2016_12_01_13_31_12_937.jpg,data/IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,data/IMG/center_2016_12_01_13_31_13_037.jpg,data/IMG/left_2016_12_01_13_31_13_037.jpg,data/IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,data/IMG/center_2016_12_01_13_31_13_177.jpg,data/IMG/left_2016_12_01_13_31_13_177.jpg,data/IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [2]:
df_collect_0 = pd.read_csv('./data_collect_0/driving_log.csv')
df_collect_1 = pd.read_csv('./data_collect_1/driving_log.csv')
#display(df_collect_0.head(n=1))
#display(df_collect_1.head(n=1))

In [3]:
frames = [df_sample, df_collect_0, df_collect_1]
df = pd.concat(frames)

In [4]:
from sklearn.cross_validation import train_test_split
df_train, df_val = train_test_split(df, test_size=0.2)

In [6]:
#print(df.shape)
#print(df.count())
#test = df_collect_1.iloc[0:4]
#for index, row in test.iterrows():
#    print(os.path.join('./', row["center"].strip()))
#    image = cv2.imread(os.path.join('./', row["center"].strip()))
    #print (image)
#    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [7]:
CENTER = "center"
LEFT = "left"
RIGHT = "right"
STEERING = "steering"
cam_corrections = { CENTER : 0, LEFT : 0.2, RIGHT : -0.2}

In [8]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
def generator(df, batch_size=32):
    num_samples = df.shape[0] #or df['center'].count()
    while 1:
        shuffle(df) # or df.sample(frac=1).reset_index(drop=True)
        for offset in range(0, num_samples, batch_size):
            df_batch = df.iloc[offset:offset + batch_size] #slice wont raise out of bound exception
            images = []
            angles = []
            for index, row in df_batch.iterrows():
                for pos in cam_corrections:
                    image = cv2.imread(os.path.join('./', row[pos].strip()))
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    angle = row[STEERING] + cam_corrections[pos]
                    images.append(image)
                    angles.append(angle)
            
            X_train = np.array(images)
            y_train = np.array(angles)
            yield X_train, y_train

In [9]:
from keras.models import Sequential
from keras.layers import Input, Lambda
from keras.layers import Dropout, Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, Cropping2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Cropping2D(cropping=((40, 40), (0, 0)), input_shape=(160, 320, 3)))
model.add(BatchNormalization())
model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(384, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1))

In [11]:
callbacks = [EarlyStopping(monitor='val_loss', patience=30, verbose=0),
             ModelCheckpoint('model.hdf5', monitor='val_loss', save_best_only=True, verbose=0)]

In [12]:
EPOCHS = 100
adam_opt=Adam(lr=5e-5)
model.compile(adam_opt, 'mse', ['accuracy'])
model_history = model.fit_generator(generator(df_train), epochs=EPOCHS,
                                    steps_per_epoch=int(df_train.shape[0]/32),
                                    validation_data=generator(df_val),
                                    validation_steps=int(df_val.shape[0]/32),
                                    callbacks = callbacks)

Epoch 1/100
332/332 [==============================] - 71s - loss: 0.0551 - acc: 0.1743 - val_loss: 0.0271 - val_acc: 0.1768
Epoch 2/100
332/332 [==============================] - 69s - loss: 0.0312 - acc: 0.1770 - val_loss: 0.0213 - val_acc: 0.1762
Epoch 3/100
332/332 [==============================] - 69s - loss: 0.0277 - acc: 0.1771 - val_loss: 0.0208 - val_acc: 0.1768
Epoch 4/100
332/332 [==============================] - 69s - loss: 0.0257 - acc: 0.1771 - val_loss: 0.0190 - val_acc: 0.1768
Epoch 5/100
332/332 [==============================] - 69s - loss: 0.0233 - acc: 0.1772 - val_loss: 0.0174 - val_acc: 0.1764
Epoch 6/100
332/332 [==============================] - 69s - loss: 0.0225 - acc: 0.1772 - val_loss: 0.0182 - val_acc: 0.1764
Epoch 7/100
332/332 [==============================] - 69s - loss: 0.0208 - acc: 0.1773 - val_loss: 0.0165 - val_acc: 0.1764
Epoch 8/100
332/332 [==============================] - 69s - loss: 0.0194 - acc: 0.1772 - val_loss: 0.0167 - val_acc: 0.1764


332/332 [==============================] - 72s - loss: 0.0055 - acc: 0.1773 - val_loss: 0.0103 - val_acc: 0.1762
Epoch 66/100
332/332 [==============================] - 72s - loss: 0.0054 - acc: 0.1772 - val_loss: 0.0104 - val_acc: 0.1762
Epoch 67/100
332/332 [==============================] - 72s - loss: 0.0055 - acc: 0.1773 - val_loss: 0.0102 - val_acc: 0.1768
Epoch 68/100
332/332 [==============================] - 71s - loss: 0.0054 - acc: 0.1772 - val_loss: 0.0106 - val_acc: 0.1772
Epoch 69/100
332/332 [==============================] - 72s - loss: 0.0053 - acc: 0.1772 - val_loss: 0.0110 - val_acc: 0.1767
Epoch 70/100
332/332 [==============================] - 72s - loss: 0.0053 - acc: 0.1773 - val_loss: 0.0106 - val_acc: 0.1762
Epoch 71/100
332/332 [==============================] - 72s - loss: 0.0052 - acc: 0.1771 - val_loss: 0.0108 - val_acc: 0.1764
Epoch 72/100
332/332 [==============================] - 72s - loss: 0.0053 - acc: 0.1772 - val_loss: 0.0106 - val_acc: 0.1764.177 -